In [3]:
cd .

/home/ducpham/Documents/Pytorch-Coding/Text_generation/Poem_Generation


## **1. Import libraries**

In [1]:
import math
import os
import re
import time
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

/home/ducpham/anaconda3/lib/python3.11/site-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/home/ducpham/anaconda3/lib/python3.11/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/home/ducpham/anaconda3/lib/python3.11/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHT

## **2. Dataset**

In [ ]:
DATASET_PATH = 'poem-datasets.csv'
df = pd.read_csv(DATASET_PATH)
df.head()

In [ ]:
df['content'][0].split('\n')


## **3. Build vectorization function**

In [ ]:
def text_normalize(text):
    text = text.strip()

    return text

def tokenizer(text):
    return text.split()

def yield_tokens(df):
    for idx, row in df.iterrows():
        yield tokenizer(row['content'])

vocab = build_vocab_from_iterator(
    yield_tokens(df),
    specials=['<unk>', '<pad>', '<sos>', '<eos>', '<eol>']
)


df['content'] = df['content'].apply(lambda x: text_normalize(x))
vocab.set_default_index(vocab['<unk>'])
vocab.get_stoi()

In [17]:
PAD_TOKEN = vocab['<pad>']
EOS_TOKEN = vocab['<eos>']

MAX_SEQ_LEN = 25

def pad_and_truncate(input_ids, max_seq_len):
    if len(input_ids) > max_seq_len:
        input_ids = input_ids[:max_seq_len]
    else:
        input_ids += [PAD_TOKEN] * (max_seq_len - len(input_ids))

    return input_ids

def vectorize(text, max_seq_len):
    input_ids = [vocab[token] for token in tokenizer(text)]
    input_ids = pad_and_truncate(input_ids, max_seq_len)

    return input_ids

def decode(input_ids):
    return [vocab.get_itos()[token_id] for token_id in input_ids]

In [20]:
print(df['content'][0].split('\n')[0])
print(vectorize(df['content'][0].split('\n')[0], 10))

Bạn xấu như chiếc bóng
[402, 1812, 34, 322, 68, 1, 1, 1, 1, 1]


## **4. Create Poem Dataset**

In [21]:
class PoemDataset(Dataset):
    def __init__(self, df, tokenizer, vectorizer, max_seq_len):
        self.tokenizer = tokenizer
        self.vectorizer = vectorizer
        self.max_seq_len = max_seq_len
        self.input_seqs, self.target_seqs, self.padding_masks = self.create_samples(df)

    def create_padding_mask(self, input_ids, pad_token_id=PAD_TOKEN):
        return [0 if token_id == pad_token_id else 1 for token_id in input_ids]

    def split_content(self, content):
        samples = []

        poem_parts = content.split('\n\n')
        for poem_part in poem_parts:
            poem_in_lines = poem_part.split('\n')
            if len(poem_in_lines) == 4:
                samples.append(poem_in_lines)

        return samples

    def prepare_sample(self, sample):
        input_seqs = []
        target_seqs = []
        padding_masks = []

        input_text = '<sos> ' + ' <eol> '.join(sample) + ' <eol> <eos>'
        input_ids = self.tokenizer(input_text)
        for idx in range(1, len(input_ids)):
            input_seq = ' '.join(input_ids[:idx])
            target_seq = ' '.join(input_ids[1:idx+1])
            input_seq = self.vectorizer(input_seq, self.max_seq_len)
            target_seq = self.vectorizer(target_seq, self.max_seq_len)
            padding_mask = self.create_padding_mask(input_seq)

            input_seqs.append(input_seq)
            target_seqs.append(target_seq)
            padding_masks.append(padding_mask)

        return input_seqs, target_seqs, padding_masks

    def create_samples(self, df):
        input_seqs = []
        target_words = []
        padding_masks = []

        for idx, row in df.iterrows():
            content = row['content']
            samples = self.split_content(content)
            for sample in samples:
                sample_input_seqs, sample_target_words, sample_padding_masks = self.prepare_sample(sample)

                input_seqs += sample_input_seqs
                target_words += sample_target_words
                padding_masks += sample_padding_masks

        input_seqs = torch.tensor(input_seqs, dtype=torch.long)
        target_words = torch.tensor(target_words, dtype=torch.long)
        padding_masks = torch.tensor(padding_masks, dtype=torch.float)

        return input_seqs, target_words, padding_masks

    def __len__(self):
        return len(self.input_seqs)

    def __getitem__(self, idx):
        input_seqs = self.input_seqs[idx]
        target_seqs = self.target_seqs[idx]
        padding_masks = self.padding_masks[idx]

        return input_seqs, target_seqs, padding_masks

TRAIN_BS = 256
train_dataset = PoemDataset(
    df=df,
    tokenizer=tokenizer,
    vectorizer=vectorize,
    max_seq_len=MAX_SEQ_LEN
)
train_loader = DataLoader(
    train_dataset,
    batch_size=TRAIN_BS,
    shuffle=False
)

In [22]:
input_seqs, target_seqs, padding_masks = next(iter(train_loader))

print(input_seqs[0])
print(target_seqs[0])
print(padding_masks[0])

tensor([2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1])
tensor([402,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.])


In [24]:
for idx in range(MAX_SEQ_LEN):
    print(decode(input_seqs[idx]))
    print(decode(target_seqs[idx]))
    break

['<sos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['Bạn', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


## **5. Create model**

In [61]:
class PositionalEncoding(nn.Module):
    def __init__(self, embedding_dims, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embedding_dims, 2) * (-math.log(10000.0) / embedding_dims))
        pe = torch.zeros(max_len, 1, embedding_dims)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        x = self.dropout(x)

        return x

class TransformerModel(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        embedding_dims: int,
        n_heads: int,
        hidden_dims: int,
        n_layers: int,
        dropout: float = 0.5
    ):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.embedding = nn.Embedding(num_embeddings=vocab_size, 
                                      embedding_dim=embedding_dims)
        self.embedding_dims = embedding_dims

        self.pos_encoder = PositionalEncoding(embedding_dims=embedding_dims, 
                                              dropout=dropout)
        
        encoder_layers = nn.TransformerEncoderLayer(
            d_model=embedding_dims,
            nhead=n_heads,
            dim_feedforward=hidden_dims, 
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer=encoder_layers, 
            num_layers=n_layers
        )
        self.linear = nn.Linear(in_features=embedding_dims, 
                                out_features=vocab_size)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.linear.bias.data.zero_()
        self.linear.weight.data.uniform_(-initrange, initrange)

    def forward(self, 
                src: torch.Tensor, 
                src_mask: torch.Tensor=None, 
                padding_mask: torch.Tensor=None):
        
        # src: [N, seq_len]
        N, seq_len = src.shape
        src = self.embedding(src) * math.sqrt(self.embedding_dims)
        src = self.pos_encoder(src)
        if src_mask is None:
            src_mask = torch.triu(input=torch.ones((seq_len, seq_len)), diagonal=1).bool()
        output = self.transformer_encoder(src, mask=src_mask, src_key_padding_mask=padding_mask)
        output = self.linear(output) # [N, seq_len, vocab_size]

        return output

In [62]:
VOCAB_SIZE = len(vocab) # 1924
EMBEDDING_DIMS = 128
HIDDEN_DIMS = 128
MAX_SEQ_LEN = 25
N_LAYERS = 2
N_HEADS = 4
DROPOUT = 0.2

device = 'cuda' if torch.cuda.is_available() else 'cpu'
input_tests = torch.randint(1, 10, (1, 10)).to(device)

model = TransformerModel(
    VOCAB_SIZE,
    EMBEDDING_DIMS,
    N_HEADS,
    HIDDEN_DIMS,
    N_LAYERS,
    DROPOUT
).to(device)

with torch.no_grad():
    output = model(input_tests)
    print(output.shape)

torch.Size([1, 10, 1924])


## **6. Training**

In [64]:
LR = 5.0
EPOCHS = 100

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.95)

In [69]:
model.train()
for epoch in range(EPOCHS):
    losses = []
    for idx, samples in enumerate(train_loader):
        input_seqs, target_seqs, padding_masks = samples
        input_seqs = input_seqs.to(device)
        target_seqs = target_seqs.to(device)
        padding_masks = padding_masks.to(device)

        output = model(input_seqs, padding_mask=padding_masks)
        output = output.permute(0, 2, 1) # [N, vocab_size, seq_len]
        loss = criterion(output, target_seqs)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        losses.append(loss.item())

    total_loss = sum(losses) / len(losses)
    print(f'EPOCH {epoch+1} \t Loss {total_loss}')
    scheduler.step()

EPOCH 1 	 Loss 540.3614836193267
EPOCH 2 	 Loss 426.22644478934154
EPOCH 3 	 Loss 460.2862359909784
EPOCH 4 	 Loss 341.24520510718935
EPOCH 5 	 Loss 273.1403125581287
EPOCH 6 	 Loss 231.195433480399
EPOCH 7 	 Loss 278.8192298525856
EPOCH 8 	 Loss 207.6133066813151
EPOCH 9 	 Loss 191.31104678199404
EPOCH 10 	 Loss 166.1860809326172
EPOCH 11 	 Loss 183.60734194800966
EPOCH 12 	 Loss 125.76590874081566
EPOCH 13 	 Loss 113.23210089547294
EPOCH 14 	 Loss 113.42255619594029
EPOCH 15 	 Loss 86.52298118954613
EPOCH 16 	 Loss 85.67820103963216
EPOCH 17 	 Loss 85.51116380237397
EPOCH 18 	 Loss 68.13210024152484
EPOCH 19 	 Loss 78.66631153651646
EPOCH 20 	 Loss 80.80233110700335
EPOCH 21 	 Loss 49.53887131100609
EPOCH 22 	 Loss 46.15447998046875
EPOCH 23 	 Loss 44.43247259230841
EPOCH 24 	 Loss 47.91933077857608
EPOCH 25 	 Loss 45.615592684064595
EPOCH 26 	 Loss 38.210173379807244
EPOCH 27 	 Loss 34.63061977568127
EPOCH 28 	 Loss 32.89529309953962
EPOCH 29 	 Loss 34.86828095572336
EPOCH 30 	 Loss

## **7. Inference**

In [66]:
def sample_with_temperature(logits, temperature=1.0):
    if temperature != 1.0:
        logits = logits / temperature

    probabilities = F.softmax(logits, dim=-1)

    sampled_index = torch.multinomial(probabilities, 1).item()

    return sampled_index

In [70]:
model.eval()
temperature = 1.2
input_text = '<sos> Anh'
input_tokens = tokenizer(input_text)
input_ids = [vocab[token] for token in input_tokens]
eos_token_id = vocab['<eos>']
generated_ids = input_ids.copy()
MAX_GENERATION_LEN = 50


for _ in range(MAX_GENERATION_LEN):
    input_tensor = torch.tensor([generated_ids], 
                                dtype=torch.long).to(device)
    with torch.no_grad():
        outputs = model(input_tensor)

    last_token_logits = outputs[0, -1, :]
    next_token_id = sample_with_temperature(last_token_logits, temperature)
    generated_ids.append(next_token_id)

    if next_token_id == eos_token_id:
        break

# Convert the generated tokens back to text
generated_text = decode(generated_ids)
generated_text = ' '.join(generated_text)
generated_text = generated_text.replace('<sos>', '')
lines = generated_text.split('<eol>')
for line in lines:
    print(''.join(line))

 Anh đồi <pad> nụ <pad> điều, Và… nhà cúng đâu 
 hỏi ròng Ngư Ta Chưa Một sao <pad> Đem <pad> Ngày bé <pad> nhìn... 
 không 
 <pad> <pad> có <pad> trên Gian đầu <pad> Tự <pad> <pad> <pad> điên <pad> Những - 
 thể nguyện 
 mẹ <pad> <pad>
